# %% [markdown]
# Model Comparison Command-Line Tool

This Jupyter notebook guides you through building a CLI tool to compare OpenAI **Base**, **Instruct**, and **Fine-tuned** models. You can:

- Input a query and select a model type
- Call the appropriate OpenAI API
- Display the response alongside a brief summary of the chosen model’s characteristics

---

# %% [markdown]
## 1. Setup and Dependencies

Install required packages:

```bash
pip install openai python-dotenv
```

Create a `.env` file (or `.env.example`) with your OpenAI API key and the fine-tuned model name:

```
OPENAI_API_KEY=your_api_key_here
FINE_TUNED_MODEL=your-fine-tuned-model-id-here
```

---

In [7]:
import os
import argparse
from dotenv import load_dotenv
import openai
import requests

# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# Load environment variables
dotenv_path = os.path.join(os.getcwd(), ".env.example")
load_dotenv(dotenv_path)

# Configure OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
hf_api_token = os.getenv("HF_API_TOKEN")
hf_model = os.getenv("HF_FINE_TUNED_MODEL")

# Mapping of model types
MODEL_MAP = {
    "base": {"provider": "openai", "model_id": "gpt-4o-mini"},
    "instruct": {"provider": "openai", "model_id": "gpt-4o-mini"}, 
    "fine-tuned": {"provider": "openai", "model_id": "gpt-4o-mini"},  # Using same model for demo
    # Note: For real fine-tuned models, you would use your custom model ID
    # "fine-tuned": {"provider": "openai", "model_id": "ft:gpt-4o-mini:your-org:your-model:abc123"},
    # or a different provider like Hugging Face
    # "fine-tuned": {"provider": "huggingface", "model_id": hf_model}
}



# Characteristics descriptions for each model type
def get_model_summary(model_type: str) -> str:
    summaries = {
        "base": (
            "Base models are pretrained on large corpora without extra instruction tuning. "
            "They are flexible but may require more prompt engineering for specific tasks."
        ),
        "instruct": (
            "Instruct models are pretrained and then tuned to follow instructions directly. "
            "They excel at tasks where you need detailed, instruction-driven outputs."
        ),
        "fine-tuned": (
            "Fine-tuned models are specialized versions of base models trained on specific datasets "
            "for particular tasks or domains. They excel at specialized use cases like content moderation, "
            "domain-specific knowledge, or particular writing styles."
        )
    }
    return summaries.get(model_type, "No summary available.")
print(get_model_summary("fine-tuned"))



Fine-tuned models are specialized versions of base models trained on specific datasets for particular tasks or domains. They excel at specialized use cases like content moderation, domain-specific knowledge, or particular writing styles.


In [8]:

# Call OpenAI for base and instruct
def call_openai_model(prompt: str, model_id: str) -> str:
    try:
        # Use chat completions for modern models like gpt-4o-mini
        if "gpt-4" in model_id or "gpt-3.5" in model_id:
            response = openai.chat.completions.create(
                model=model_id,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150,
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        else:
            # Use legacy completions for other models
            response = openai.completions.create(
                model=model_id,
                prompt=prompt,
                max_tokens=150,
                temperature=0.7
            )
            return response.choices[0].text.strip()
    except Exception as e:
        return f"Error calling model {model_id}: {str(e)}"

# Call Hugging Face for fine-tuned
def call_hf_model(prompt: str, model_id: str) -> str:
    url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {hf_api_token}"}
    payload = {"inputs": prompt, "options": {"use_cache": False}}
    res = requests.post(url, headers=headers, json=payload)
    res.raise_for_status()
    data = res.json()
    # Hugging Face returns a list of generation dicts
    if isinstance(data, dict) and data.get("error"):
        raise ValueError(data["error"])
    first = data[0]
    return first.get("generated_text", first.get("text", ""))

# Main API switcher
def call_model(query: str, model_type: str) -> str:
    config = MODEL_MAP.get(model_type)
    if not config:
        raise ValueError(f"Model type '{model_type}' is not configured.")
    provider = config["provider"]
    model_id = config["model_id"]

    if provider == "openai":
        return call_openai_model(query, model_id)
    elif provider == "huggingface":
        return call_hf_model(query, model_id)
    else:
        raise ValueError(f"Unknown provider '{provider}' for model type '{model_type}'.")





In [9]:
# Five different examples for BASE model
print("=== BASE MODEL EXAMPLES ===")
print("\n1. Creative Writing:")
print(call_model("Once upon a time in a digital world", "base"))

print("\n2. Technical Explanation:")
print(call_model("Explain quantum computing in simple terms", "base"))

print("\n3. Problem Solving:")
print(call_model("How to solve a Rubik's cube", "base"))

print("\n4. Philosophical Question:")
print(call_model("What is the meaning of life", "base"))

print("\n5. Data Analysis:")
print(call_model("Analyze the trends in renewable energy", "base"))

print("\n" + "="*50)

# Five different examples for INSTRUCT model
print("=== INSTRUCT MODEL EXAMPLES ===")
print("\n1. Email Writing:")
print(call_model("Write a friendly email to a client about project delays.", "instruct"))

print("\n2. Recipe Instructions:")
print(call_model("Give me step-by-step instructions to make chocolate chip cookies", "instruct"))

print("\n3. Educational Content:")
print(call_model("Explain the process of photosynthesis for a 10-year-old", "instruct"))

print("\n4. Business Plan:")
print(call_model("Create a brief business plan for a coffee shop", "instruct"))

print("\n5. Travel Guide:")
print(call_model("Provide a travel itinerary for 3 days in Paris", "instruct"))

print("\n" + "="*50)

# Five different examples for FINE-TUNED model
print("=== FINE-TUNED MODEL EXAMPLES ===")
print("\n1. Content Moderation:")
print(call_model("This post contains inappropriate language", "fine-tuned"))

print("\n2. Safety Assessment:")
print(call_model("Is this message safe for children", "fine-tuned"))

print("\n3. Policy Compliance:")
print(call_model("Does this content violate community guidelines", "fine-tuned"))

print("\n4. Risk Analysis:")
print(call_model("Analyze potential risks in this user message", "fine-tuned"))

print("\n5. Content Classification:")
print(call_model("Classify this text as appropriate or inappropriate", "fine-tuned"))


=== BASE MODEL EXAMPLES ===

1. Creative Writing:
Once upon a time in a digital world, there existed a vibrant realm known as ByteLand. This was a place where data flowed like rivers, and the landscape was dotted with pixelated mountains and shimmering code forests. The inhabitants of ByteLand were not ordinary beings but friendly digital creatures called Bytes, each representing a different type of data: TextBytes, ImageBytes, AudioBytes, and even VideoBytes.

The Bytes lived in harmony, sharing information and collaborating to create wondrous applications that enhanced the lives of users in the real world. They built towering structures made of algorithms and formed lush gardens of interactive websites. The air buzzed with the energy of connections, as every Byte worked tirelessly to ensure that information was accessible and enjoyable.

However, peace in

2. Technical Explanation:
Sure! Quantum computing is a type of computing that uses the principles of quantum mechanics, which is 

In [10]:
# Display model summaries
print("\n" + "="*60)
print("MODEL TYPE SUMMARIES")
print("="*60)

for model_type in ["base", "instruct", "fine-tuned"]:
    print(f"\n{model_type.upper()} MODEL:")
    print("-" * 20)
    print(get_model_summary(model_type))
    print()



MODEL TYPE SUMMARIES

BASE MODEL:
--------------------
Base models are pretrained on large corpora without extra instruction tuning. They are flexible but may require more prompt engineering for specific tasks.


INSTRUCT MODEL:
--------------------
Instruct models are pretrained and then tuned to follow instructions directly. They excel at tasks where you need detailed, instruction-driven outputs.


FINE-TUNED MODEL:
--------------------
Fine-tuned models are specialized versions of base models trained on specific datasets for particular tasks or domains. They excel at specialized use cases like content moderation, domain-specific knowledge, or particular writing styles.

